In [73]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 10)
from data.loader import load_data
from data import loader
from matplotlib import pyplot as plt
import numpy as np
import utils
import matplotlib.image as mpimg
from model.models import LRCN, model_dict
from data.loader import load_data
import time
import cv2

import torch

Populating the interactive namespace from numpy and matplotlib


In [74]:
BEST_LRCN_CHECKPOINT = '/deep/group/aihc-bootcamp-winter2018/medical-imaging/mr_knee_abnormality/models/allison_mr_knee_2d_sequential_newdata_uniform/1519081402/val0.4986041188240051_train0.4832608997821808_epoch30'

In [32]:
class Argument():
    def __init__(self, params):
        self.pretrained = params['pretrained']
        self.datadir = params['datadir']
        self.rundir = params['rundir']
        self.model = params['model']
        self.seq = params['seq']
        self.optimizer = params['optimizer']
        self.learning_rate = params['learning_rate']
        self.weight_decay = params['weight_decay']
        self.dropout = params['dropout']
        self.batch_size = params['batch_size']
        self.epochs = params['epochs']
        self.max_patience = params['max_patience']
        self.verbose = params['verbose']
        self.plot = params['plot']
        self.rgb = params['rgb']
        self.scale = params['scale']
        self.horizontal_flip = params['horizontal_flip']
        self.rotate = params['rotate']
        self.reverse = params['reverse']
        self.workers = params['workers']
        self.toy = params['toy']
        self.extension = params['extension']
        self.fix_num_slices = params['fix_num_slices']
        self.num_slices = params['num_slices']
        self.shift = params['shift']
        self.weighted_loss = params['weighted_loss']
        self.hidden_dim = params['hidden_dim']

In [33]:
_args = {
    "datadir": "/deep/group/aihc-bootcamp-winter2018/medical-imaging/mr_knee_abnormality/data-normalized",
    "rundir": "/deep/group/aihc-bootcamp-winter2018/medical-imaging/mr_knee_abnormality/models/allison_mr_knee_2d_sequential/1519081402",
    "model": "lrcn",
    "pretrained": True,
    "weighted_loss": False,
    "seq": "lstm",
    "hidden_dim": 16,
    "optimizer": "adam",
    "learning_rate": 0.0005,
    "weight_decay": 0.001,
    "dropout": 0.1,
    "batch_size": 1,
    "epochs": 50,
    "max_patience": 8,
    "verbose": False,
    "plot": False,
    "rgb": True,
    "scale": 256,
    "horizontal_flip": True,
    "rotate": 25,
    "reverse": True,
    "workers": 8,
    "toy": False,
    "extension": "npy",
    "fix_num_slices": True,
    "num_slices": 28,
    "shift" : 0,
    "weighted_loss" : False
}
args = Argument(_args)

In [67]:
def load_model(args, num_classes):
    def hook_feature(module, input, output):
        features_blobs[0] = output.data.cpu().numpy()


    start = time.time()
    if args.model == 'lrcn':
        model = model_dict[args.model](args, num_classes, args.hidden_dim)
        state_dict = torch.load(BEST_LRCN_CHECKPOINT)
        model.load_state_dict(state_dict)
        finalconv_name = 'features'
        model.cnn._modules.get(finalconv_name).register_forward_hook(hook_feature)
    else:
        raise ValueError("Visualization only implemented for lrcn, received ", args.model)
    
    model.eval()

    done = time.time()
    elapsed = done - start
    print("Loading time: ", elapsed)
    return model

In [69]:
def returnCAM(feature_conv, weight_softmax, size_upsample, class_idx):
    # generate the class activation maps upsample to 256x256
    # size_upsample = (224, 224)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [70]:
def infer(model, image):

    start = time.time()

    params = list(model.parameters())
    weight_softmax = np.squeeze(params[-2].data.numpy())
    img_int = (image.data.numpy() * 255).astype('uint8')
    img_int = np.moveaxis(img_int, 0, -1)

    #img_variable = Variable(imageunsqueeze(0))
    logit = model(image)

    h_x = F.softmax(logit).data.squeeze()

    CAMs = [returnCAM(features_blobs[0], weight_softmax, img_int.shape[:2], [i]) for i in range(NUM_CLASSES)]

    done = time.time()
    elapsed = done - start
    print("Inference time: ", elapsed)
    return h_x, CAMs

In [78]:
def get_heatmap(CAM, img):
    height, width = img.shape[:2]
    cam = cv2.resize(CAM,(width, height))
    cam = sklearn.preprocessing.minmax_scale(cam)
    cmap = plt.get_cmap('gnuplot2')
    cam = cmap(cam) * 255
    heatmap = cam[:, :, :3]
    # heatmap = cv2.applyColorMap( cv2.COLORMAP_AUTUMN)
    # print(img.max())
    result = heatmap * 0.3 + img * 0.5

    return result

In [79]:
def infer_example(model,numpy_image):
    # Expects numpy image (H, W ,C) with pixel values in range [0,255)
    orig_img = np.copy(numpy_image)
    img = transform_image(numpy_image)
    prediction, CAMs = infer(model, img)

    # render the CAM and output
    results = []

    # if shap_flag is True:
    #   results = get_shap_maps(img, model)
    # else:
    for CAM in CAMs:
        result = get_heatmap(CAM[0], orig_img)
        results.append(result)
    return prediction.numpy(), results

In [80]:
def get_labels():
    return ['Abnormal']

In [34]:
train_loader, valid_loader, test_loader, rad_loader = load_data(args)

In [77]:
model = load_model(args, train_loader.dataset.num_classes)

Loading time:  0.35059380531311035


In [89]:
img, label = valid_loader.dataset[0]

RuntimeError: There are less number of slices than num_slices

In [58]:
model.cnn._modules.get('features')

Sequential(
  (conv0): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu0): ReLU(inplace)
  (pool0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU(inplace)
      (conv.1): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (relu.2): ReLU(inplace)
      (conv.2): Conv2d (128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
      (relu.1): ReLU(inplace)
      (conv.1): Conv2d (96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affin

In [41]:
state_dict = torch.load('/deep/group/aihc-bootcamp-winter2018/medical-imaging/mr_knee_abnormality/models/allison_mr_knee_2d_sequential_newdata_inner/1519081402/val0.4986041188240051_train0.4832608997821808_epoch30')

In [43]:
state_dict.keys()

odict_keys(['cnn.features.conv0.weight', 'cnn.features.norm0.weight', 'cnn.features.norm0.bias', 'cnn.features.norm0.running_mean', 'cnn.features.norm0.running_var', 'cnn.features.denseblock1.denselayer1.norm.1.weight', 'cnn.features.denseblock1.denselayer1.norm.1.bias', 'cnn.features.denseblock1.denselayer1.norm.1.running_mean', 'cnn.features.denseblock1.denselayer1.norm.1.running_var', 'cnn.features.denseblock1.denselayer1.conv.1.weight', 'cnn.features.denseblock1.denselayer1.norm.2.weight', 'cnn.features.denseblock1.denselayer1.norm.2.bias', 'cnn.features.denseblock1.denselayer1.norm.2.running_mean', 'cnn.features.denseblock1.denselayer1.norm.2.running_var', 'cnn.features.denseblock1.denselayer1.conv.2.weight', 'cnn.features.denseblock1.denselayer2.norm.1.weight', 'cnn.features.denseblock1.denselayer2.norm.1.bias', 'cnn.features.denseblock1.denselayer2.norm.1.running_mean', 'cnn.features.denseblock1.denselayer2.norm.1.running_var', 'cnn.features.denseblock1.denselayer2.conv.1.weight

In [44]:
model.load_state_dict(state_dict)